In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna
from optuna import Trial, visualization
from hepml.core import plot_regression_tree


In [18]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
climateregions__climateregion


In [19]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [20]:
data = train_df[main_attrs + target]

In [21]:
def train_model_by_X_y(_X, _y):
    skf = KFold(n_splits=5, random_state=2048, shuffle=True)
    skf.get_n_splits(_X, _y)

    train_index_list = []
    test_index_list = []
    train_x_list = []
    val_x_list = []
    train_y_list = []
    val_y_list = []

    for i, (train_index, test_index) in enumerate(skf.split(_X, _y)):
        train_index_list.append(train_index)
        test_index_list.append(test_index)

        train_x_list.append(_X[train_index, :])
        val_x_list.append(_X[test_index, :])
        train_y_list.append(_y[train_index])
        val_y_list.append(_y[test_index])    
   

    cat_models = []
    for i in range(5):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i],
        clf = CatBoostRegressor(iterations=5000, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=4999, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models


In [22]:
def cal_train_pred(models, _X):
    preds = []
    for model in models:
        preds.append(model.predict(_X))
    s = 0
    for pred in preds:
        s += pred
    s = s/5
    return s

In [23]:
n_estimators = 3
Xtest = test_df[main_attrs].values
Xtrain = train_df[main_attrs].values

In [24]:
pred_list = []
res = 0
best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm
y = data[target[0]].values
X = Xtrain
for i in tqdm(range(n_estimators)):
    cat_models = train_model_by_X_y(X, y)
    s1 = cal_train_pred(cat_models, X)
    
    pred_s1 = cal_train_pred(cat_models, Xtest)
    pred_list.append(pred_s1)
    data['Cat {} prediction'.format(i)] = s1
    data['Cat {} residual'.format(i)] = y - s1
    res += pred_s1
    
    y_residual = data['Cat {} residual'.format(i)]
    residual_models = train_model_by_X_y(X, y_residual)
    
    s2 = cal_train_pred(residual_models, X)
    pred_s2 = cal_train_pred(residual_models, Xtest)
    pred_list.append(pred_s2)
    res += pred_s2
    
    if len(pred_list) >= 2:
        cos_sim = dot(y_best, res)/(norm(y_best)*norm(res))
        print("Mean:", res.mean())
        print("cos_sim with best submission:", cos_sim)
        
    
    data["Cat {} residual prediction".format(i)] = s2

    data["Cat {} residual prediction residual".format(i)] = data['Cat {} residual'.format(i)] - data["Cat {} residual prediction".format(i)]
    
    del pred_s2, s2, residual_models, y_residual, pred_s1, s1, cat_models
    
    y = data["Cat {} residual prediction residual".format(i)]
    

  0%|                                                                                                | 0/3 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 6344.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2589355	test: 9.2381618	best: 9.2381618 (0)	total: 28.4ms	remaining: 2m 21s
4999:	learn: 0.3260566	test: 0.3538970	best: 0.3538970 (4999)	total: 25.5s	remaining: 0us
bestTest = 0.3538969515
bestIteration = 4999


Learning rate set to 0.072699
0:	learn: 9.2521715	test: 9.2693331	best: 9.2693331 (0)	total: 7.05ms	remaining: 35.2s
4999:	learn: 0.3249096	test: 0.3512136	best: 0.3512136 (4999)	total: 1m 22s	remaining: 0us
bestTest = 0.3512136365
bestIteration = 4999


Learning rate set to 0.072699
0:	learn: 9.2481240	test: 9.2814895	best: 9.2814895 (0)	total: 74ms	remaining: 6m 9s
4999:	learn: 0.3250658	test: 0.3520980	best: 0.3520980 (4999)	total: 2m 38s	remaining: 0us
bestTest = 0.3520980197
bestIteration = 4999


Learning rate set to 0.072699
0:	learn: 9.2553172	test: 9.2517667	best: 9.2517667 (0)	total: 76.3ms	remaining: 6m 21s
4999:	learn: 0.3270225	test: 0.3548023	best: 0.3548023 (4999)	total: 2m 51s	remaining: 0us
bestTest = 0.3548023369
bestIteration = 4999


...

Learning rate set to 0.072699
0:	learn: 0.1312219	test: 0.1309808	best: 0.1309808 (0)	total: 5.91ms	remaining: 29.5s
4999:	learn: 0.1175647	test: 0.1254491	best: 0.1254491 (4999)	total: 23s	remaining: 0us
bestTest = 0.1254490668
bestIteration = 4999


Learning rate set to 0.072699
0:	learn: 0.1311414	test: 0.1313060	best: 0.1313060 (0)	total: 6.03ms	remaining: 30.2s
4999:	learn: 0.1174386	test: 0.1257789	best: 0.1257789 (4999)	total: 22.7s	remaining: 0us
bestTest = 0.1257789193
bestIteration = 4999


Learning rate set to 0.072699
0:	learn: 0.1311147	test: 0.1314046	best: 0.1314046 (0)	total: 7.52ms	remaining: 37.6s
4999:	learn: 0.1174977	test: 0.1259086	best: 0.1259086 (4999)	total: 22.9s	remaining: 0us
bestTest = 0.1259086203
bestIteration = 4999


100%|███████████████████████████████████████████████████████████████████████████████████████| 3/3 [24:50<00:00, 496.85s/it]

Mean: 14.79028359830829
cos_sim with best submission: 0.9999762966693929


In [28]:
test_s = 0
for pred in pred_list:
    test_s += pred


In [30]:
df = pd.read_csv('./sample_solution.csv')
s = test_s
df['contest-tmp2m-14d__tmp2m'] = s
df.to_csv('submission_corr_important_base_catboost_v1.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

cos_sim with best submission: 0.9999762966693929
